thread = threading.Thread(
    target=функция,      # Что выполнять
    args=(аргументы,),   # Аргументы для функции
    kwargs={словарь},    # Именованные аргументы
    daemon=True/False    # Демон-поток (объясню ниже)
)

1. Создание и запуск потока

In [ ]:
import threading
import time

def fnc(name):
    print(f"Поток {name} начал работу")
    time.sleep(2)
    print(f"Поток {name} завершил работу")

# Самый простой способ
put = threading.Thread(target=fnc, args=("Вася",))
put.start()  # Запускаем поток

put.join()

Пример 1: Несколько потоков

In [ ]:
import threading
import time

def download_file(file_name):
    print(f"📥 Начинаю загрузку {file_name}")
    time.sleep(3)  # Имитация загрузки
    print(f"✅ Загружено: {file_name}")

# Создаем потоки для 3 файлов
pul = []
files = ["фильм.mp4", "музыка.mp3", "документ.pdf"]

for file in files:
    t = threading.Thread(target=download_file, args=(file,))
    pul.append(t)
    t.start()  # Запускаем сразу

# Ждем завершения ВСЕХ потоков
for t in pul:
    t.join()

print("🎉 Все файлы загружены!")

Пример 2: Поток с возвратом результата

In [ ]:
import threading

def hig_arifmetics(number):
    result = number * number
    return result

# Чтобы получить результат из потока
result_pul = None

def pul_result(n):
    global result_pul
    result_pul = hig_arifmetics(n)

t = threading.Thread(target=pul_result, args=(50000000,))
t.start()
t.join()

print(f"Результат: {result_pul}")  # 25

🔒 Синхронизация (очень важно!)

In [ ]:
import threading

counter = 0

def add():
    global counter
    for _ in range(100000):
        counter += 1

# Запускаем 2 потока
t1 = threading.Thread(target=add)
t2 = threading.Thread(target=add)

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Счетчик: {counter}")  # ❌ Не будет 200000!

In [ ]:
import threading

counter = 0
lock = threading.Lock()  # Создаем замок

def add():
    global counter
    for _ in range(100000):
        with lock:  # Захватываем замок
            counter += 1
        # Замок автоматически освобождается

t1 = threading.Thread(target=add)
t2 = threading.Thread(target=add)

t1.start()
t2.start()
t1.join()
t2.join()

print(f"Счетчик: {counter}")  # ✅ Всегда 200000

👻 Демон-потоки

In [ ]:
import threading
import time


def work_while():
    while True:
        print("Я работаю...")
        time.sleep(1)

# Демон-поток завершится с основной программой
dem = threading.Thread(target=work_while, daemon=True)
dem.start()

print("Основная программа завершается, демон тоже умрет")

In [1]:
import requests
import time
import os
import threading

os.makedirs("images", exist_ok=True)

urls = [
    "https://picsum.photos/400/300?random=1",
    "https://picsum.photos/400/300?random=2",
    "https://picsum.photos/400/300?random=3",
    "https://picsum.photos/400/300?random=4",
    "https://picsum.photos/400/300?random=5"
]

def скачать_картинку(url):
    имя = url.split("=")[-1] + ".jpg"
    путь = f"images/{имя}"
    print(f"Начинаю скачивать {имя}...")
    response = requests.get(url)
    with open(путь, "wb") as f:
        f.write(response.content)
    print(f"✅ {имя} готова!")

# === БЕЗ ПОТОКОВ ===
print("=== Скачивание ПО ОЧЕРЕДИ ===")
start = time.time()
for url in urls:
    скачать_картинку(url)
end = time.time()
print(f"🕗 Время без потоков: {end - start:.2f} секунд\n")

# === С ПОТОКАМИ ===
print("=== Скачивание В ПОТОКАХ ===")
start = time.time()
потоки = []
for url in urls:
    t = threading.Thread(target=скачать_картинку, args=(url,))
    потоки.append(t)
    t.start()

for t in потоки:
    t.join()

end = time.time()
print(f"⚡ Время с потоками: {end - start:.2f} секунд")

=== Скачивание ПО ОЧЕРЕДИ ===
Начинаю скачивать 1.jpg...
✅ 1.jpg готова!
Начинаю скачивать 2.jpg...
✅ 2.jpg готова!
Начинаю скачивать 3.jpg...
✅ 3.jpg готова!
Начинаю скачивать 4.jpg...
✅ 4.jpg готова!
Начинаю скачивать 5.jpg...
✅ 5.jpg готова!
🕗 Время без потоков: 2.23 секунд

=== Скачивание В ПОТОКАХ ===
Начинаю скачивать 1.jpg...
Начинаю скачивать 2.jpg...
Начинаю скачивать 3.jpg...
Начинаю скачивать 4.jpg...
Начинаю скачивать 5.jpg...
✅ 3.jpg готова!
✅ 1.jpg готова!
✅ 5.jpg готова!
✅ 4.jpg готова!
✅ 2.jpg готова!
⚡ Время с потоками: 0.17 секунд


In [3]:
import cv2
import threading
import time
import numpy as np

# Флаг для остановки потока
остановить_поток = False

def показывать_окно():
    global остановить_поток
    # Создаём "холст" — чёрное изображение 400x300
    изображение = np.zeros((300, 400, 3), dtype=np.uint8)

    while not остановить_поток:
        # Меняем цвет фона каждую секунду (для демонстрации)
        t = int(time.time()) % 3
        if t == 0:
            изображение[:] = [0, 0, 0]      # чёрный
        elif t == 1:
            изображение[:] = [0, 255, 0]    # зелёный
        else:
            изображение[:] = [255, 0, 0]    # красный

        cv2.imshow("Моё окно", изображение)
        
        # Ждём 1 мс — обязательно для обновления окна OpenCV!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            остановить_поток = True  # нажали 'q' — выходим

    cv2.destroyAllWindows()

# Запускаем поток с OpenCV
поток_окна = threading.Thread(target=показывать_окно)
поток_окна.daemon = True  # умрёт, если основная программа завершится
поток_окна.start()

# Основная программа делает своё дело
print("Основная программа запущена!")
for i in range(10):
    print(f"Работаю... шаг {i+1}")
    time.sleep(1)  # имитация полезной работы

# Готовимся завершить
остановить_поток = True
поток_окна.join()  # ждём завершения потока
print("Готово!")

Основная программа запущена!
Работаю... шаг 1
Работаю... шаг 2
Работаю... шаг 3
Работаю... шаг 4
Работаю... шаг 5
Работаю... шаг 6
Работаю... шаг 7
Работаю... шаг 8
Работаю... шаг 9
Работаю... шаг 10
Готово!


In [ ]:
import cv2
import threading
import time
import numpy as np
from queue import Queue, Empty

# Очередь для передачи кадров
очередь_кадров = Queue(maxsize=1)

def показывать_кадры():
    while True:
        try:
            кадр = очередь_кадров.get(timeout=1)
            if кадр is None:  # сигнал завершения
                break
            cv2.imshow("Видео", кадр)
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break
        except Empty:
            continue
    cv2.destroyAllWindows()

# Запускаем поток показа
threading.Thread(target=показывать_кадры, daemon=True).start()

# Основная программа генерирует "видео"
for i in range(100):
    # Создаём кадр с движущимся кругом
    кадр = np.zeros((400, 600, 3), dtype=np.uint8)
    x = i * 6 % 600
    cv2.circle(кадр, (x, 200), 30, (0, 255, 255), -1)
    
    # Кладём в очередь (если очередь полная — ждём)
    очередь_кадров.put(кадр)
    time.sleep(0.05)  # ~20 кадров/сек

# Завершение
очередь_кадров.put(None)
time.sleep(0.1)

In [ ]:
import threading
import time



def func_tes(number, counter):
    global counter1, counter2
    for i in range(0, number):
        if counter == 1:
            time.sleep(2)
        if counter == 2:
            time.sleep(1)
        print(i)
    if counter == 1:  
       counter1 = 5 * number
    else:
        counter2 = 5 * number


counter1 = 0
counter2 = 0  

flow1 = threading.Thread(target=func_tes, args=(200, 1))
flow2 = threading.Thread(target=func_tes, args=(300, 2))

flow1.start()
flow2.start()

flow1.join()
flow2.join()

print(counter1 + counter2)

0
0
1
2
1
3
4
2
5
6
3
7
8
4
9
10
5
11
12
6
13
14
7
15
16
8
17
18
9
19
20
10
21
22
11
23
24
12
25
26
13
27
28
14
29
30
15
31
32
16
33
34
17
35
36
18
37
38
19
39
40
20
41
42
21
43
44
22
45
46
23
47
48
24
49
50
25
51
52
26
53
54
27
55
56
28
57
58
29
59
60
30
61
62
31
63
64
32
65
66
33
67
68
34
69
70
35
71
72
36
73
74
37
75
76
38
77
78
39
79
80
40
81
82
41
83
84
42
85
86
43
87
88
44
89
90
45
91
92
46
93
94
47
95
96
48
97
98
49
99
100
50
101
102
51
103
104
52
105
106
53
107
108
54
109
110
55
111
112
56
113
114
57
115
116
58
117
118
59
119
120
60
121
122
61
123
124
62
125
126
63
127
128
64
129
130
65
131
132
66
133
134
67
135
136
68
137
138
69
139
140
70
141
142
71
143
144
72
145
146
73
147
148
74
149
150
75
151
152
76
153
154
77
155
156
78
157
158
79
159
160
80
161
162
81
163
164
82
165
166
83
167
168
84
169
170
85
171
172
86
173
174
87
175
176
88
177
178
89
179
180
